In [1]:
"""class AnimalData:
    def __init__(self,root,transforms):
        self.root=root
        self.transforms=transforms
        classes=sort(list(os.listdir(root)))
        images=[]
        for c in classes:
            if c=='Label':
                continue
            for img in sort(list(os.listdir(c))):
                images.append(root+c+img)
        self.images=images
        label_dir=[c+'/Label' for c in classes]
        labels=[]
        for c in classes:
            for label in sort(list(os.listdir(label_dir))):
                labels.append(root+label_dir+label)
        self.labels=labels
    def __getitem__(self,idx):
        img =Image.open(self.img[idx]).convert("RGB")
        with open(self.labels,'r') as f:
            content= f.read()
            lines=contents.split(' ')
            for i in range(1, len(lines)):
                lines[i]=int(lines[i])
        
        """
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import tqdm 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_labels0=[]
test_labels0=[]
train_imgs0=[]
test_imgs0=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels0.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs0.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels0.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png')or image.endswith('jpg'):
                test_imgs0.append(dirname+'/'+image)


In [31]:
print(len(train_imgs0))
print(len(train_labels0))
print(len(test_imgs0))
print(len(test_labels0))
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
for i in range(len(train_labels0)):
    if i%5==0:
        
        train_labels.append(train_labels0[i])
        train_imgs.append(train_imgs0[i])
for i in range(len(test_labels0)):
        test_labels.append(test_labels0[i])
        test_imgs.append(test_imgs0[i])
print(len(train_imgs))
print(len(train_labels))
print(len(test_imgs))
print(len(test_labels))


14620
14620
6864
6864
2924
2924
6864
6864


In [4]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [6]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model_instance_mobilenet(num_classes):
    
    # load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")

    # replace the classifier with a new one, that has
    # num_classes which is user-defined
    num_classes = len(classes)+1  # 1 class (person) + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


In [7]:
from engine import train_one_epoch, evaluate
import utils

train_data=AnimalDataset(get_transform(train=False),train_imgs,train_labels)
val_data=AnimalDataset(get_transform(train=False),train_imgs,train_labels)
test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person


# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function

model =get_model_instance_mobilenet(len(classes)+1)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=7,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 50

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
    # update the learning rate
    torch.save(model.state_dict(), "./model/fasterRCNN/fasterRCNN_mobilenet.pt")
    
    print('saved dict')
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model,val_loader, device=device)
    evaluate(model,test_loader, device=device)


Epoch: [0]  [  0/731]  eta: 0:42:55  lr: 0.000012  loss: 4.3723 (4.3723)  loss_classifier: 3.4679 (3.4679)  loss_box_reg: 0.8497 (0.8497)  loss_objectness: 0.0067 (0.0067)  loss_rpn_box_reg: 0.0480 (0.0480)  time: 3.5233  data: 0.3310  max mem: 1598


KeyboardInterrupt: 

In [ ]:
decode=[]
for c in classes:
    decode.append(

In [53]:
import time
from PIL import Image, ImageDraw,ImageFont
# Load the model
i=2499
model =get_model_instance_mobilenet(len(classes)+1)
model.load_state_dict(torch.load('./model/fasterRCNN/fasterRCNN_mobilenet.pt'))
model.to('cuda')
model.eval()
test_data=AnimalDataset(get_transform(train=False),train_imgs[i:i+1],train_labels[i:i+1])
# Load a batch of images
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

timecount=0
for imgs,target in test_loader:
    images = list(img.to('cuda') for img in imgs)
    torch.cuda.synchronize()
    start_time = time.time()
    # Run the model on the images
    with torch.no_grad():
        
        outputs = model(images)
    timecount+=time.time()-start_time
    # Measure the end time
    max_pred=torch.argmax(outputs[0]['scores'])
    boxes = [outputs[0]['boxes'][max_pred]   ]  
    # Load the image using PIL
    

    # Draw the boxes on the image
    
        

    # Display the image
    img = Image.open(train_imgs[i])

    # Create a drawing object
    draw = ImageDraw.Draw(img)

    # Define the font to use for the labels
    font = ImageFont.truetype("/usr/share/fonts/truetype/abyssinica/AbyssinicaSIL-Regular.ttf", 16)

    # Define the bounding boxes and labels
    
    labels = [classes[outputs[0]['labels'][max_pred]-1]]
    print(labels)
    print(boxes)
    # Draw the bounding boxes and labels
    for box, label in zip(boxes, labels):
        draw.rectangle(box.tolist(), outline="red", width=2)
        draw.text((box), label, font=font)

    # Display the image
    img.show()
# Measure the start time


# Calculate the inference time per image

['Cat']
[tensor([ 66.2551, 233.8878, 456.1722, 608.0673], device='cuda:0')]


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [9]:
model=get_model_instance_mobilenet(len(classes)+1)
model.to

In [10]:
test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)

In [17]:
import torch
import cv2

# Load the trained YOLOv3 model


# Load the image
img = cv2.imread('data/val/Cat/00b6be538644be0b.jpg')

# Preprocess the input image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB color space
img = torch.from_numpy(img.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0) # Normalize and convert to tensor

# Pass the preprocessed image to the model to get the output prediction
with torch.no_grad():
    pred = model(img)

# Apply non-maximum suppression to remove redundant bounding boxes
pred = pred[0].xyxy[0]

# Visualize the output prediction by drawing bounding boxes around the detected objects
img = cv2.cvtColor(img[0].numpy().transpose(1, 2, 0), cv2.COLOR_RGB2BGR) # Convert back to BGR color space
for box in pred:
    cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

cv2.imshow('Prediction', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

AssertionError: targets should not be none when in training mode